In [6]:
from scipy.interpolate import (
    LinearNDInterpolator,
    NearestNDInterpolator,
    CloughTocher2DInterpolator,
    Rbf,
)

import matplotlib.pyplot as plt
import firedrake
import firedrake_adjoint

from firedrake import Constant, cos, sin

import numpy as np
from numpy import pi as π
from numpy import random

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import os, sys

currentdir = os.path.dirname(os.path.realpath('__file__'))

import argparse

parser = argparse.ArgumentParser(description='Estimate q using pyadjoint with a given number of point samples of u_true and chosen method. Expects to find a Firedrake checkpoint file \'true-fields.h5\' in the import directory.')
parser.add_argument('num_points', type=int, help='The number of points to sample from u_true. Points and measurements be identified from \'observed-data.h5\' or created and saved to it.')
parser.add_argument('method', help="The method to use: one of point-cloud, nearest, linear, clough-tocher, or gaussian")
try:
    args = parser.parse_args()
    num_points = args.num_points
    method = args.method
except:
    import warnings
    warnings.warn(f'Failed to parse arguments. Defaulting to num_points = 4 and method = point-cloud')
    num_points = 100
    method = 'point-cloud'

methods = ['point-cloud', 'nearest', 'linear', 'clough-tocher', 'gaussian']

# If running as notebook use default of 4 points and method 'point-cloud'
if method not in methods:
    import warnings
    warnings.warn(f'Got unexpected method argument {method} defaulting to point-cloud')
    method = 'point-cloud'
    
print(f"Running with {num_points} points and method {method}")

seed = 1729

Running with 100 points and method point-cloud


usage: ipykernel_launcher.py [-h] num_points method
ipykernel_launcher.py: error: argument num_points: invalid int value: '/Users/andrew/Library/Jupyter/runtime/kernel-c015ec64-9f86-42b0-8f07-27472bd66fde.json'
<ipython-input-6-86f412619182>:36: UserWarning: Failed to parse arguments. Defaulting to num_points = 4 and method = point-cloud
  warnings.warn(f'Failed to parse arguments. Defaulting to num_points = 4 and method = point-cloud')


In [7]:
mesh = firedrake.UnitSquareMesh(32, 32)

# Solution Space
V = firedrake.FunctionSpace(mesh, family='CG', degree=2)

# q (Control) Space
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)

In [8]:
q_true = firedrake.Function(V, name='q_true')
u_true = firedrake.Function(V, name='u_true')
filename = os.path.join(currentdir, 'true-fields')
with firedrake.DumbCheckpoint(filename, mode=firedrake.FILE_READ) as chk:
    chk.load(q_true, name='q_true')
    chk.load(u_true, name='u_true')
    
print("Have fake q_true and u_true")

Have fake q_true and u_true


In [9]:
import h5py

filename = os.path.join(currentdir, 'observed-data.h5')

try:
    # Load if available
    with h5py.File(filename, 'r') as file:
        xs = file[f"xs_{num_points}"][:]
        u_obs_vals = file[f"u_obs_vals_{num_points}"][:]
        σ = firedrake.Constant(file[f"sigma_{num_points}"])
        print(f"Loaded xs, u_obs_vals and sigma for {num_points} points.")
except (OSError, KeyError) as e:
    # Generate
    np.random.seed(0)
    xs = np.random.random_sample((num_points, 2))
    xs[:,1]=.9
    signal_to_noise = 20
    U = u_true.dat.data_ro[:]
    u_range = U.max() - U.min()
    σ = firedrake.Constant(u_range / signal_to_noise)
    generator = random.default_rng(seed)
    ζ = generator.standard_normal(len(xs))
    u_obs_vals = np.array(u_true.at(xs)) + float(σ) * ζ
    # Save
    with h5py.File(filename, 'a') as file:
        file.create_dataset(f"xs_{num_points}", data=xs)
        file.create_dataset(f"u_obs_vals_{num_points}", data=u_obs_vals)
        file.create_dataset(f"sigma_{num_points}", data=σ.values()[0])
    print(f"Generated and saved xs, u_obs_vals and sigma for {num_points} points.")

Generated and saved xs, u_obs_vals and sigma for 100 points.
